# Algoritmo Genético para treinamento da RNA

Importe os pacotes necessários

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from imutils import paths
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from keras import optimizers
import numpy as np
from numpy import array, dtype
import argparse
import random
import pickle
import cv2
import os
import livelossplot
import collections
import asyncio
import sys


c:\users\johnp\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Estrutura do Algoritmo Genético

## Operações para definir os pesos da RNA

In [2]:
def inserePesosConv2d(lista, j, k, h):
	qtdPesos = 32
	g = 0
	while(g < qtdPesos):
		peso = random.uniform(-1.0, 1.0)
		lista[j][k][h].append(peso)
		g += 1
	return lista

def montaConv2d_1():
	camada = list()
	lista = list() ## no primeiro caso deve ter 3 posições, no segundo caso deve ter 32 posições

	descendente = 2 ## quantidade de elementos na camada

	i = 0
	while(i < descendente):
		lista = []
		if( i == 0):
			tamanhoSubCamada = 3
			j = 0
			while(j < tamanhoSubCamada):
				subLista = list()
				lista.append(subLista)

				tamanhoSubSubCamada = 3
				k = 0
				while(k < tamanhoSubSubCamada):
					subsubLista = list()
					lista[j].append(subsubLista)

					tamanhoSubSubSubcamada = 3
					h = 0
					while(h < tamanhoSubSubSubcamada):
						subsubsubLista = list()
						lista[j][k].append(subsubsubLista)

						lista = inserePesosConv2d(lista, j, k, h)

						h += 1
					k += 1
				j += 1
						
		elif(i == 1):
			qtdPesos = 32
			j = 0
			while(j < qtdPesos):
				lista.append(0.0)
				j += 1
		else:
			print("Não pode ter mais que duas posições")
		
		camada.append(np.array(lista, dtype='float32'))
		i += 1
	return camada



def montaCamada(arraysNaCamada, qtdPorArray):
	camada = list()
	return camada 

def montaBatch_normalization(arraysNaCamada, qtdPorArray):
	camada = list()
	if(not arraysNaCamada == 0):
		i = 0
		lista = None
		
		while(i < arraysNaCamada):
			lista = list()
			
			pesoAtual = 0
			while(pesoAtual < qtdPorArray[i]):
				if(i == 0 or i == 3):
					numero = 1.0
				else:
					numero = 0.0

				lista.append(numero)
				pesoAtual += 1

			camada.append(np.array(lista, dtype='float32'))
			i = i + 1

	return camada 

def montaPesoNaDensa(lista):
	qtdPesos = 32768
	g = 0
	while(g < qtdPesos):
		sub = []
		peso = random.uniform(-1.0, 1.0)
		sub.append(peso)
		lista.append(sub)
		g += 1
	return lista

def montaDensa(arraysNaCamada, qtdPorArray):
	camada = list()
	if(not arraysNaCamada == 0):
		i = 0
		lista = None
		
		while(i < arraysNaCamada):
			lista = []	
			DEFAULT = 0.0
			if(i == 0):
				result = montaPesoNaDensa(lista)
			elif(i == 1):
				peso = DEFAULT
				lista.append(peso)
			else:
				print("Nao era pra cair aqui")

			i = i + 1
			camada.append(np.array(lista, dtype='float32'))
	
	return camada 

arraysNaCamada = None
qtdPorArray = None

## Inicia um Indivíduo

In [3]:
def init_individual():
    individual = []
    array0 = np.array([],dtype='float32')
    array1 = montaConv2d_1() 
    array2 = montaCamada(0, []) # é vazio
    array3 = montaBatch_normalization(4, [32,32,32,32])
    array4 = montaCamada(0, []) # é vazio
    array5 = montaCamada(0, []) # é vazio
    array6 = montaDensa(2, [32768, 1])
    array7 = montaBatch_normalization(4, [1,1,1,1])
    individual.append(array0)
    individual.append(array1)
    individual.append(array2)
    individual.append(array3)
    individual.append(array4)
    individual.append(array5)
    individual.append(array6)
    individual.append(array7)
    return individual

## Inicia uma nova População

In [4]:
def init_new_population(population_size):
    
    new_population = []
    
    iterator = 0
    while (iterator < population_size):
        individual = init_individual()
        new_population.append(individual)
        iterator = iterator + 1
    
    return(new_population)

## Execução da RNA

In [5]:
def execute_rna(list0, list1, list2, list3, list4, list5, list6, list7):

    dataset = 'dataset'

    EPOCHS = 3
    INIT_LR = 1e-3
    MOMENTUM = 0.8
    BS = 5
    IMAGE_DIMS = (96, 96, 3)
    modelname = 'blood.model'
    label = 'lb.pickle'

    data = []
    labels = []

    imagePaths = list(paths.list_images(dataset))

    for imagePath in imagePaths:
        # carrega a imagem, pré-processa e armazena na lista de dados
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
        image = img_to_array(image)
        data.append(image)

        # extrai o rótulo da classe do caminho da imagem e atualiza a lista de rótulos
        label = imagePath.split(os.path.sep)[-2]
        labels.append(label)

    data = np.array(data, dtype="float") / 255.0
    labels = np.array(labels)
    print("[INFO] data matrix: {:.2f}MB".format(
        data.nbytes / (1024 * 1000.0)))

    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)

    (trainX, testX, trainY, testY) = train_test_split(data,
        labels, test_size=0.2, shuffle=False )

    aug = ImageDataGenerator()
    width=IMAGE_DIMS[1]
    height=IMAGE_DIMS[0]
    depth=IMAGE_DIMS[2]
    classes=len(lb.classes_)

    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    if K.image_data_format() == "channels_first":
        inputShape = (depth, height, width)
        chanDim = 1

    # CONV => RELU => POOL
    model.add(Conv2D(32, (3, 3), padding="same",
        input_shape=inputShape, use_bias=True))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3, 3)))

    #set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(1))
    model.add(BatchNormalization())


#     print('original')
#     print('#########################################################')
#     for layer in model.layers:
#         weights = layer.get_weights()
#     #     print(weights)

    # layer0 = layer[0].get_weights()
    #print(model.layers)
#     print(model.get_layer(Conv2D,0).get_weights())
#     print(model.get_layer(Conv2D,1).get_weights())
#     print(model.get_layer(Conv2D,2).get_weights())
#     print(model.get_layer(Conv2D,3).get_weights())
#     print(model.get_layer(Conv2D,4).get_weights())
#     print(model.get_layer(Conv2D,5).get_weights())
#     print(model.get_layer(Conv2D,6).get_weights())
#     print(model.get_layer(Conv2D,7).get_weights())

    #model.summary()    

    # Classificador
    model.add(Dense(classes))
    model.add(Activation("tanh")) #tanh <- Tangente hiperbólica
    opt = optimizers.SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS, momentum=MOMENTUM, nesterov=True)
    
    model.get_layer(Conv2D,0).set_weights(list0)
    model.get_layer(Conv2D,1).set_weights(list1)
    model.get_layer(Conv2D,2).set_weights(list2)
    model.get_layer(Conv2D,3).set_weights(list3)
    model.get_layer(Conv2D,4).set_weights(list4)
    model.get_layer(Conv2D,5).set_weights(list5)
    model.get_layer(Conv2D,6).set_weights(list6)
    model.get_layer(Conv2D,7).set_weights(list7)
    
#     print(model.get_layer(Conv2D,1).get_weights()) 
#     print(model.get_layer(Conv2D,1).get_weights())
#     print(model.get_layer(Conv2D,2).get_weights())
#     print(model.get_layer(Conv2D,3).get_weights())
#     print(model.get_layer(Conv2D,4).get_weights())
#     print(model.get_layer(Conv2D,5).get_weights())
#     print(model.get_layer(Conv2D,6).get_weights())
#     print(model.get_layer(Conv2D,7).get_weights())
    
    model.compile(loss="categorical_crossentropy", optimizer=opt,
        metrics=["accuracy"])
    plot_losses = livelossplot.PlotLossesKeras()
    H = model.fit_generator(
        aug.flow(trainX, trainY, batch_size=BS),
        validation_data=(testX, testY),
        steps_per_epoch=len(trainX) // BS,
        epochs=EPOCHS,
        #callbacks=[plot_losses],
        verbose=1
    )
    train_loss = H.history["loss"]
    val_loss = H.history["val_loss"]
    train_acc = H.history["acc"]
    val_acc = H.history["val_acc"]

    fitness = (sum(train_acc)/len(train_acc))
    return fitness

## Testa cada indivíduo da População e obtém o fitness

In [6]:
def get_fitness(population):
    #inicia a matrix 2x6 da população
    population_matrix = [[0]*2 for i in range(6)]
    
    #Obtém fitness do indivíduo 1
    population_matrix[0][0] = population[0]
    population_matrix[0][1] = execute_rna(population[0][0], population[0][1], population[0][2], population[0][3], population[0][4], population[0][5], population[0][6], population[0][7])

    
    #Obtém fitness do indivíduo 2
    population_matrix[1][0] = population[1]
    population_matrix[1][1] = execute_rna(population[1][0], population[1][1], population[1][2], population[1][3], population[1][4], population[1][5], population[1][6], population[1][7])

    #Obtém fitness do indivíduo 3
    population_matrix[2][0] = population[2]
    population_matrix[2][1] = execute_rna(population[2][0], population[2][1], population[2][2], population[2][3], population[2][4], population[2][5], population[2][6], population[2][7])

    #Obtém fitness do indivíduo 4
    population_matrix[3][0] = population[3]
    population_matrix[3][1] = execute_rna(population[3][0], population[3][1], population[3][2], population[3][3], population[3][4], population[3][5], population[3][6], population[3][7])

    #Obtém fitness do indivíduo 5
    population_matrix[4][0] = population[4]
    population_matrix[4][1] = execute_rna(population[4][0], population[4][1], population[4][2], population[4][3], population[4][4], population[4][5], population[4][6], population[4][7])

    #Obtém fitness do indivíduo 6
    population_matrix[5][0] = population[5]
    population_matrix[5][1] = execute_rna(population[5][0], population[5][1], population[5][2], population[5][3], population[5][4], population[5][5], population[5][6], population[5][7])

    return(population_matrix)

## Seleciona os 2 melhores indivíduos da população

In [7]:
def select_best_individuals(population):
    sorted_population = sorted(population_fitness, key=lambda population_fitness: population_fitness[1], reverse=True)
    sorted_population.pop(5)
    sorted_population.pop(4)
    sorted_population.pop(3)
    sorted_population.pop(2)
    
    return(sorted_population)

## Troca características entre os melhores indivíduos

In [8]:
def crossover_individuals(n2_population):
    """
    Slices both dna1 and dna2 into two parts at a random index within their
    length and merges them. Both keep their initial sublist up to the crossover
    index, but their ends are swapped.
    """
#     #indivíduo 1
#    print(n2_population[0][0][1][0][0][0][0])
#     print(n2_population[0][0][1][0][0][0][1])
#     print(n2_population[0][0][1][0][0][0][2])
#     print(n2_population[0][0][1][0][0][1][0])
#     print(n2_population[0][0][1][0][0][1][1])
#     print(n2_population[0][0][1][0][0][1][2])
#     print(n2_population[0][0][1][0][0][2][0])
#     print(n2_population[0][0][1][0][0][2][1])
#     print(n2_population[0][0][1][0][0][2][2])
#     print(n2_population[0][0][1][0][1][0][0])
#     print(n2_population[0][0][1][0][1][0][1])
#     print(n2_population[0][0][1][0][1][0][2])
#     print(n2_population[0][0][1][0][1][1][0])
#     print(n2_population[0][0][1][0][1][1][1])
#     print(n2_population[0][0][1][0][1][1][2])
#     print(n2_population[0][0][1][0][1][2][0])
#     print(n2_population[0][0][1][0][1][2][1])
#     print(n2_population[0][0][1][0][1][2][2])
#     print(n2_population[0][0][1][0][2][0][0])
#     print(n2_population[0][0][1][0][2][0][1])
#     print(n2_population[0][0][1][0][2][0][2])
#     print(n2_population[0][0][1][0][2][1][0])
#     print(n2_population[0][0][1][0][2][1][1])
#     print(n2_population[0][0][1][0][2][1][2])
#     print(n2_population[0][0][1][0][2][2][0])
#     print(n2_population[0][0][1][0][2][2][1])
#     print(n2_population[0][0][1][0][2][2][2])
#    print(n2_population[0][0][6][0])
#     #indivíduo 2
#    print(n2_population[1][0][1][0][0][0][0])
#     print(n2_population[1][0][1][0][0][0][1])
#     print(n2_population[1][0][1][0][0][0][2])
#     print(n2_population[1][0][1][0][0][1][0])
#     print(n2_population[1][0][1][0][0][1][1])
#     print(n2_population[1][0][1][0][0][1][2])
#     print(n2_population[1][0][1][0][0][2][0])
#     print(n2_population[1][0][1][0][0][2][1])
#     print(n2_population[1][0][1][0][0][2][2])
#     print(n2_population[1][0][1][0][1][0][0])
#     print(n2_population[1][0][1][0][1][0][1])
#     print(n2_population[1][0][1][0][1][0][2])
#     print(n2_population[1][0][1][0][1][1][0])
#     print(n2_population[1][0][1][0][1][1][1])
#     print(n2_population[1][0][1][0][1][1][2])
#     print(n2_population[1][0][1][0][1][2][0])
#     print(n2_population[1][0][1][0][1][2][1])
#     print(n2_population[1][0][1][0][1][2][2])
#     print(n2_population[1][0][1][0][2][0][0])
#     print(n2_population[1][0][1][0][2][0][1])
#     print(n2_population[1][0][1][0][2][0][2])
#     print(n2_population[1][0][1][0][2][1][0])
#     print(n2_population[1][0][1][0][2][1][1])
#     print(n2_population[1][0][1][0][2][1][2])
#     print(n2_population[1][0][1][0][2][2][0])
#     print(n2_population[1][0][1][0][2][2][1])
#     print(n2_population[1][0][1][0][2][2][2])
#    print(n2_population[1][0][6][0])
    print(n2_population[1][0][6][0].shape)
    new_individual11 = np.array([], dtype='float32')
    new_individual12 = np.array([], dtype='float32')
    new_individual21 = np.array([], dtype='float32')
    new_individual22 = np.array([], dtype='float32')
    
    #pos=int(random.random()*len(n2_population[0][0][6][0]))
    pos = 2
    for i in range(0,len(n2_population[0][0][1][0])):
        for j in range(0,len(n2_population[0][0][1][0][0])):
            for k in range(0,len(n2_population[0][0][1][0][0][0])):
                new_individual11 = (n2_population[0][0][1][0][i][j][k][:pos])
                new_individual12 = (n2_population[1][0][1][0][i][j][k][pos:])
                new_individual21 = (n2_population[1][0][1][0][i][j][k][:pos])
                new_individual22 = (n2_population[0][0][1][0][i][j][k][pos:])
                n2_population[0][0][1][0][i][j][k] = np.append(new_individual11, new_individual12)
                n2_population[1][0][1][0][i][j][k] = np.append(new_individual21, new_individual22)
    
    print('print1')
    print(n2_population[1][0][6][0].shape)          
    new_individual11 = np.array([], dtype='float32')
    new_individual12 = np.array([], dtype='float32')
    new_individual21 = np.array([], dtype='float32')
    new_individual22 = np.array([], dtype='float32')
    
    new_individual11 = (n2_population[0][0][6][0][:pos])
    print(new_individual11)
    new_individual12 = (n2_population[1][0][6][0][pos:])
    new_individual21 = (n2_population[1][0][6][0][:pos])
    new_individual22 = (n2_population[0][0][6][0][pos:])
    n2_population[0][0][6][0] = np.append(new_individual11, new_individual12)
    n2_population[1][0][6][0] = np.append(new_individual21, new_individual22)
    
    print(n2_population[1][0][6][0].shape)
    print(n2_population[1][0][6][0])
    return n2_population
#     new1 = []
#     new2 = []
#     list1 = [1,2,3,4]
#     list2 = [11,22,33,44]
#     pos=int(random.random()*len(list1))
#     print(pos)
#     print(list1, list2)
#     new1.extend(list1[:pos])
#     new1.extend(list2[pos:])
#     new2.extend(list2[:pos])
#     new2.extend(list1[pos:])
#     list1 = new1
#     list2 = new2
#     print(list1)
#     print(list2)

In [9]:
def random_float():
    """
    Return a random number between -1 and 1
    """
    return random.uniform(-1, 1)

In [10]:

def mutate(crossed_population):
    """
    For each gene in the DNA, there is a 1/mutation_chance chance that it will be
    switched out with a random character. This ensures diversity in the
    population, and ensures that is difficult to get stuck in local minima.
    """
    mutations = 0
    mutation_chance = 100
    index = 0
    
    for i in range(0,len(crossed_population[0][0][1][0])):
        for j in range(0,len(crossed_population[0][0][1][0][0])):
            for k in range(0,len(crossed_population[0][0][1][0][0][0])):
                if int(random.random()*mutation_chance) == 1:
                    k = random_float()
                    mutations = mutations + 1
                else:
                    #nothing to do
                    ('nem era pra entrar aqui')
                index = index + 1
    
    index2 = 0

    for m in crossed_population[0][0][6][0]:
        if int(random.random()*mutation_chance) == 1:
            m = random_float()
            mutations = mutations + 1 
        else:
            #nothing to do
            ('nem era pra entrar aqui2')
        index2 = index2 + 1

    print('Ocorreram',mutations, 'mutações')
    return crossed_population

In [11]:
def repopulate(mutated_population):
    
    population = []
    population.append(mutated_population[0][0])
    population.append(mutated_population[1][0])
    iterator = 0
    while (iterator < 4):
        individual = init_individual()
        population.append(individual)
        iterator = iterator + 1

    return(population)

In [21]:
population = init_new_population(6)
population_fitness = get_fitness(population)
print(population_fitness[0][1])
print(population_fitness[1][1])
print(population_fitness[2][1])
print(population_fitness[3][1])
print(population_fitness[4][1])
print(population_fitness[5][1])


[INFO] data matrix: 4.32MB
Epoch 1/3
3/3 [==============================] - 1s 366ms/step - loss: 10.3495 - acc: 0.4182 - val_loss: 16.1181 - val_acc: 0.5000
Epoch 2/3
3/3 [==============================] - 0s 17ms/step - loss: 9.4416 - acc: 0.3455 - val_loss: 16.1181 - val_acc: 0.5000
Epoch 3/3
3/3 [==============================] - 0s 28ms/step - loss: 4.5299 - acc: 0.2667 - val_loss: 12.0886 - val_acc: 0.5000
[INFO] data matrix: 4.32MB
Epoch 1/3
3/3 [==============================] - 1s 377ms/step - loss: nan - acc: 0.4909 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/3
3/3 [==============================] - 0s 38ms/step - loss: nan - acc: 0.6364 - val_loss: nan - val_acc: 0.0000e+00
Epoch 3/3
3/3 [==============================] - 0s 29ms/step - loss: nan - acc: 0.0667 - val_loss: nan - val_acc: 0.0000e+00
[INFO] data matrix: 4.32MB
Epoch 1/3
3/3 [==============================] - 1s 407ms/step - loss: 5.9774 - acc: 0.2000 - val_loss: 0.7275 - val_acc: 0.7500
Epoch 2/3
3/3 [=======

In [28]:
top2_population = select_best_individuals(population_fitness)

In [29]:
print(top2_population[0][1])
print(top2_population[0][0][6][0].shape)
print(top2_population[1][1])
print(top2_population[1][0][6][0].shape)
    
population_crossed = crossover_individuals(top2_population)
print(population_crossed[1][0][6])

population_mutated = mutate(population_crossed)

new_population = repopulate(population_mutated)

0.3252525291960649
(32768,)
0.2707070738378197
(32768,)
(32768,)
print1
(32768,)
[-0.15969312 -0.9055921 ]
(32768,)
[-0.7688903  -0.61763525 -0.80464107 ... -0.7803236  -0.47959656
  0.86692905]
[array([-0.7688903 , -0.61763525, -0.80464107, ..., -0.7803236 ,
       -0.47959656,  0.86692905], dtype=float32), array([0.], dtype=float32)]
Ocorreram 323 mutações


In [30]:
print(new_population[0][6][0][0])
print(new_population[1][6][0][0])
print(new_population[2][6][0][0])

convert_dense = new_population[2][6][0][0]
convert_dense2 = new_population[3][6][0][0]

for i in range(0,len(convert_dense)):
    convert_dense[i] = new_population[0][6][0][i]

for j in range(0, len(convert_dense2)):
    convert_dense2[j] = new_population[1][6][0][j]

new_population[0][6][0] = convert_dense
new_population[1][6][0] = convert_dense2
print(new_population[0][6][0][0])
print(new_population[1][6][0][0])

-0.15969312
-0.7688903
[-0.77805334]
-0.15969312
-0.7688903
